In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
import httpx

from urllib.parse import urljoin

from will_it_saturate.servers import (
    FastAPIUvicornServer,
    NginxDockerServer,
    DjangoGunicornWSGIServer,
)
from will_it_saturate.hosts import Host
from will_it_saturate.epochs import Epoch

from will_it_saturate.results import Result
from will_it_saturate.files import BenchmarkFile
from will_it_saturate.control.client import ControlClient
from will_it_saturate.repositories import SqliteRepository
from will_it_saturate.clients import HttpxClient, AioHttpClient, WrkClient

<IPython.core.display.Javascript object>

# Remote Benchmark

In [ ]:
# dont_test

byte, file_size, duration = 8, 10 ** 7, 3
epoch = Epoch(file_size=file_size, duration=duration)

control_host = Host(name="localhost", port=8001)
control_client = ControlClient(host=control_host)

server = control_client.get_or_create_server(FastAPIUvicornServer())
print(server)
benchmark_client = HttpxClient(name="httpx")

files = control_client.get_or_create_files(epoch)
print(files[:10])

epoch.urls = [server.file_to_url(file) for file in files]

result = Result(
    server=server.name,
    client=benchmark_client.name,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

protocol='http' name='fastAPI/uvicorn' host='localhost' port=5001
[BenchmarkFile(number=0, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='4a45eb0c5580122823126fc30477ce08', creator_name='filesystem'), BenchmarkFile(number=1, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='22b964829c3cd80f9627861eee97ad6f', creator_name='filesystem'), BenchmarkFile(number=2, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='2313682c8f900d9011cf3557356c0b75', creator_name='filesystem'), BenchmarkFile(number=3, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='9f09900cb8d58d1e470e087682f385b2', creator_name='filesystem'), BenchmarkFile(number=4, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='2e55ad3afc9

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result.elapsed = benchmark_client.measure(epoch)

measure


<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result.bytes_per_second

52186936.92073767

<IPython.core.display.Javascript object>

In [ ]:
def run_epoch(server, client, epoch):
    control_base = "http://localhost:8001/"
    control_epochs = urljoin(control_base, "epochs/")
    r = httpx.post(control_epochs, json=epoch.dict())

    for bfile in r.json()["files"]:
        kwargs = {k: v for k, v in bfile.items() if k != "creator"}
        epoch.files.append(BenchmarkFile(**kwargs))

    control_servers = urljoin(control_base, "servers/")
    r = httpx.post(control_servers, json=server.dict())
    server = server.__class__(**r.json())

    epoch.urls = [server.file_to_url(file) for file in epoch.files]

    result = Result(
        server=server.name,
        client=client.name,
        file_size=epoch.file_size,
        complete_size=epoch.complete_size,
    )

    result.elapsed = client.measure(epoch)
    return result

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

byte, file_size, duration = 8, 10 ** 7, 3
epoch = Epoch(file_size=file_size, duration=duration)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

server = FastAPIUvicornServer()
client = HttpxClient(name="httpx")

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result = run_epoch(server, client, epoch)

measure


<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result.bytes_per_second

52162649.75030322

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte
duration = 3
file_size = 10 ** 7

epoch = Epoch(file_size=file_size, duration=duration)

control_base = "http://localhost:8001/"
control_epochs = urljoin(control_base, "epochs/")

r = httpx.post(control_epochs, json=epoch.dict())

for bfile in r.json()["files"]:
    kwargs = {k: v for k, v in bfile.items() if k != "creator"}
    epoch.files.append(BenchmarkFile(**kwargs))


server = FastAPIUvicornServer()
control_servers = urljoin(control_base, "servers/")
r = httpx.post(control_servers, json=server.dict())
server = FastAPIUvicornServer(**r.json())

epoch.urls = [server.file_to_url(file) for file in epoch.files]

client = HttpxClient(name="httpx")


result = Result(
    server=server.name,
    client=client.name,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

result.elapsed = client.measure(epoch)
result

measure


Result(server='fastAPI/uvicorn', client='httpx', file_size=10000000, elapsed=6.916868163999999, complete_size=375000000)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result.bytes_per_second

54215288.06226934

<IPython.core.display.Javascript object>

# Local Benchmark

In [ ]:
# dont_test

byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte
duration = 3
file_size = 10 ** 7


# server = DjangoGunicornWSGIServer()
server = FastAPIUvicornServer()
# client = AioHttpClient(name="aiohttp")
client = HttpxClient(name="httpx")
# repository = SqliteRepository.build_repository("foo.db")

epoch = Epoch(file_size=file_size, duration=duration)

epoch.create_files()

result = Result(
    server=server.name,
    client=client.name,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

if not server.started:
    server.start

result.elapsed = client.measure(epoch)

measure


IndexError: list index out of range

<IPython.core.display.Javascript object>